In [2]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")
#Crear base de datos
db = client["mi_base"]


In [3]:
#Crear la colección
collection = db["mi colección"]
#Inserto documento de prueba
collection.insert_one({"mensaje": "Hola, MongoDB desde Python!"})

print("Base de datos y colección creadas")


Base de datos y colección creadas


In [1]:
pip install pymongo

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import json

# Ruta completa al archivo en Descargas
ruta_json = r"C:\Users\HP\Downloads\nobel_winners.json"

# Cargar JSON
with open(ruta_json, "r", encoding="utf-8") as file:
    data = json.load(file)

# Insertar en MongoDB
if isinstance(data, list):
    collection.insert_many(data)  
else:
    collection.insert_one(data)  

print("Datos insertados en MongoDB ✅")


Datos insertados en MongoDB ✅


In [21]:
documents = list(collection.find())

In [22]:
#Diccionarios limpios:
updated_files = {}

In [23]:
for doc in documents:
    for key, value in doc.items():
        if isinstance(value, bytes):
            updated_files[key] = value.decode("utf-8")
        else:
            updated_files[key] = str(value) #TODOS LOS VALORES LO HARÉ STRING

In [24]:
for doc in documents:
    for key, value in doc.items():
        if value == "" or value is None:
            updated_files[key] = "NaN" #NO PONGO VALOR VACIO EN SI, SINO UN STRING


In [25]:
import re

for doc in documents:
    if "name" in doc and isinstance(doc["name"], str):
        updated_files["name"] = re.sub(r"\*", "", doc["name"]) #Regular expresion para quitar asteriscos en name

# Eliminar los individuos repetidos. (verificar que de veras sea duplicado ya que podría haber ganado 2 veces el nobel en distinto año)

In [26]:
for doc in documents:    
    if "name" in doc and "year" in doc:
        duplicate_count = collection.count_documents({"name": doc["name"], "year": doc["year"]})
        if duplicate_count > 1:
            updated_files["is_duplicate"] = True  # Marcar duplicados en lugar de eliminarlos

In [ ]:
import numpy as np
from datetime import datetime

#Ciclamos los demás a la vez
for doc in documents:

    #Este creo que es mejor hacerlo al consultar que con MongoDB:
    # collection.find().sort([("name", 1), ("country", 1), ("year", 1)])

    # 6) Rellenar categoría faltante 
    if "category" in doc and doc["category"] is None and "year" in doc:
        same_year_doc = collection.find_one({"year": doc["year"], "category": {"$ne": None}}) #Uso arbitrariamente a otro ganador
        if same_year_doc:
            updated_files["category"] = same_year_doc["category"]

    # 7) Rellenar el género de los registros faltantes 
    if "gender" in doc and doc["gender"] is None:
        updated_files["gender"] = "Unknown"

    # 8) Convertir fechas de nacimiento y muerte en formato `numpy.datetime64`
    for date_field in ["date_of_birth", "date_of_death"]:
        if date_field in doc and isinstance(doc[date_field], str):
            try:
                updated_files[date_field] = np.datetime64(datetime.strptime(doc[date_field], "%Y-%m-%d"))
            except ValueError:
                updated_files[date_field] = None  # Si no hay fecha

    # 9) Eliminar premios que no fueron dados a personas (si no tiene un `name` válido)
    if "name" not in doc or not isinstance(doc["name"], str):
        collection.delete_one({"_id": doc["_id"]})
        continue  # Pasamos al siguiente documento para evitar hacer un update innecesario

    # Aplicar cambios en la base de datos
    if updated_files:
        collection.update_one({"_id": doc["_id"]}, {"$set": updated_files})

# Crear índice para ordenar automáticamente las consultas
collection.create_index([("name", 1), ("country", 1), ("year", 1)])